Load_data

In [51]:
import numpy as np

train_array = np.load('train_array.npy')
test_array = np.load('test_array.npy')
val_array = np.load('val_array.npy')
train_labels = np.load('train_labels.npy')
val_labels = np.load('val_labels.npy')

In [41]:
print(val_array[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  31 136 101
  63 189 254 255 254 254 160  27   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  21 154 217 253 253 253 253 253 253 253 253 253 152
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  45 253 253
 253 253 233 249 253 253 253 253 253 221  37   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   2 163 163  65  65  41  59  65  65  87 234
 253 253 135   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  22   0   0   0   0   0   0   0   0 128 253 253 13

Model construction

In [52]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

for i in range(len(train_labels)):
    if train_labels[i] == -1: train_labels[i] = 0
for i in range(len(val_labels)):
    if val_labels[i] == -1: val_labels[i] = 0


# normalization
train_array = train_array / 255.0
val_array = val_array / 255.0
test_array = test_array / 255.0

# reshape
train_array = train_array.reshape((train_array.shape[0], 28, 28, 1))
val_array = val_array.reshape((val_array.shape[0], 28, 28, 1))
test_array = test_array.reshape((test_array.shape[0], 28, 28, 1))

# CNN
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # layers.Conv2D(64, (3,3), activation='relu'),
    # layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = 'accuracy')

model.fit(train_array, train_labels, epochs = 5, batch_size = 50)

test_loss, test_acc = model.evaluate(val_array, val_labels)

print('Test accuracy:', test_acc)

Epoch 1/5
800/800 [==============================] - 4s 5ms/step - loss: 0.0178 - accuracy: 0.9912
Epoch 2/5
800/800 [==============================] - 4s 5ms/step - loss: 8.8128e-06 - accuracy: 1.0000
Epoch 3/5
800/800 [==============================] - 4s 5ms/step - loss: 2.2779e-06 - accuracy: 1.0000
Epoch 4/5
800/800 [==============================] - 4s 5ms/step - loss: 1.0846e-06 - accuracy: 1.0000
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 4.2824e-07 - accuracy: 1.0000
Test accuracy: 1.0


In [47]:
print(train_labels[:30])
print(model.predict(train_array[:30]))

[0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1]
1/1 [==============================] - 0s 67ms/step
[[1.2704610e-03]
 [1.0269368e-03]
 [9.9999982e-01]
 [9.9999958e-01]
 [7.9022668e-04]
 [7.7354710e-04]
 [9.9999881e-01]
 [9.9997866e-01]
 [1.0303344e-03]
 [9.7857241e-04]
 [2.3379782e-03]
 [9.9999845e-01]
 [7.7354710e-04]
 [9.9997407e-01]
 [1.4511710e-03]
 [7.7354710e-04]
 [7.7354710e-04]
 [8.0430479e-04]
 [7.7354710e-04]
 [8.3836872e-04]
 [7.7391573e-04]
 [9.9999827e-01]
 [9.9997681e-01]
 [8.1715506e-04]
 [9.9998248e-01]
 [7.7354710e-04]
 [9.9998820e-01]
 [9.9990195e-01]
 [9.9999619e-01]
 [9.9909788e-01]]


In [54]:
y_pred = model.predict(test_array)  

output = []

for i in range(len(y_pred)):
    if y_pred[i] > 0.5: output.append(1)
    else: output.append(-1)

input_file = 'cig_interview_data/test.txt'
output_file = 'cig_interview_data/test_CNN.txt'

with open(input_file, 'r') as f:
    lines = f.readlines()

assert len(lines) == len(y_pred)

modified_lines = [line.rstrip('0\n') + str(output[i]) + '\n' for i, line in enumerate(lines)]

with open(output_file, 'w') as f:
    f.writelines(modified_lines)

250/250 [==============================] - 0s 2ms/step
